# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [5]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("../Weather_Database/Csv/WeatherPy_database.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Laguna,US,38.4210,-121.4238,63.32,53,0,5.75,clear sky
1,1,Norman Wells,CA,65.2820,-126.8329,14.04,85,75,3.44,light snow
2,2,Albany,US,42.6001,-73.9662,63.23,88,100,3.00,light rain
3,3,Ushuaia,AR,-54.8000,-68.3000,49.66,57,40,26.46,scattered clouds
4,4,Babanka,UA,48.7097,30.4483,44.24,88,4,5.26,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [8]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature criteria for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature criteria for your trip? "))

What is the minimum temperature criteria for your trip? 75
What is the maximum temperature criteria for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [9]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]  
                                    
# Display sample data
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Namatanai,PG,-3.6667,152.4333,78.89,80,100,4.09,overcast clouds
14,14,Puerto Ayora,EC,-0.7393,-90.3518,78.76,81,69,11.99,broken clouds
17,17,Butaritari,KI,3.0707,172.7902,81.36,76,71,14.63,broken clouds
18,18,Sorong,ID,-0.8833,131.2500,80.51,77,97,1.77,overcast clouds
28,28,Sinnamary,GF,5.3833,-52.9500,79.86,84,50,6.38,scattered clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [10]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna() 

# Display sample data
clean_travel_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Namatanai,PG,-3.6667,152.4333,78.89,80,100,4.09,overcast clouds
14,14,Puerto Ayora,EC,-0.7393,-90.3518,78.76,81,69,11.99,broken clouds
17,17,Butaritari,KI,3.0707,172.7902,81.36,76,71,14.63,broken clouds
18,18,Sorong,ID,-0.8833,131.2500,80.51,77,97,1.77,overcast clouds
28,28,Sinnamary,GF,5.3833,-52.9500,79.86,84,50,6.38,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
655,655,Marabba,SD,12.3500,32.1833,80.92,18,77,12.15,broken clouds
659,659,Kavaratti,IN,10.5669,72.6420,82.08,60,100,3.15,overcast clouds
660,660,Fomboni,KM,-12.2800,43.7425,77.76,76,0,8.28,clear sky
664,664,Fria,GN,10.4167,-13.5833,77.22,74,30,2.46,scattered clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [11]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy() 

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng
11,Namatanai,PG,78.89,overcast clouds,-3.6667,152.4333
14,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518
17,Butaritari,KI,81.36,broken clouds,3.0707,172.7902
18,Sorong,ID,80.51,overcast clouds,-0.8833,131.2500
28,Sinnamary,GF,79.86,scattered clouds,5.3833,-52.9500


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [12]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = None

# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Namatanai,PG,78.89,overcast clouds,-3.6667,152.4333,None
14,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,None
17,Butaritari,KI,81.36,broken clouds,3.0707,172.7902,None
18,Sorong,ID,80.51,overcast clouds,-0.8833,131.2500,None
28,Sinnamary,GF,79.86,scattered clouds,5.3833,-52.9500,None


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [13]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [14]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"] 
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json() 
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Namatanai,PG,78.89,overcast clouds,-3.6667,152.4333,Seagulls Inn Namatani
14,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
17,Butaritari,KI,81.36,broken clouds,3.0707,172.7902,Isles Sunset Lodge
18,Sorong,ID,80.51,overcast clouds,-0.8833,131.2500,Waigo Splash Hotel
28,Sinnamary,GF,79.86,scattered clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [17]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Namatanai,PG,78.89,overcast clouds,-3.6667,152.4333,Seagulls Inn Namatani
14,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
17,Butaritari,KI,81.36,broken clouds,3.0707,172.7902,Isles Sunset Lodge
18,Sorong,ID,80.51,overcast clouds,-0.8833,131.2500,Waigo Splash Hotel
28,Sinnamary,GF,79.86,scattered clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [18]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [19]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [20]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Seagulls Inn Namatani</dd>\n<dt>City</dt><dd>Namatanai</dd>\n<dt>Country</dt><dd>PG</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 78.89 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Finch Bay Galapagos Hotel</dd>\n<dt>City</dt><dd>Puerto Ayora</dd>\n<dt>Country</dt><dd>EC</dd>\n<dt>Current Weather</dt><dd>broken clouds and 78.76 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Isles Sunset Lodge</dd>\n<dt>City</dt><dd>Butaritari</dd>\n<dt>Country</dt><dd>KI</dd>\n<dt>Current Weather</dt><dd>broken clouds and 81.36 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Waigo Splash Hotel</dd>\n<dt>City</dt><dd>Sorong</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 80.51 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>ALETHEIA Appartement Sinnamary</dd>\n<dt>City</dt><dd>Sinnamary</dd>\n<dt>Country</dt><dd>GF</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 79.86 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>H

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [21]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
11,-3.6667,152.4333
14,-0.7393,-90.3518
17,3.0707,172.7902
18,-0.8833,131.2500
28,5.3833,-52.9500
29,-22.9661,-42.0278
31,5.9485,80.5353
36,-21.2000,-175.2000
47,21.4612,-71.1419
54,-3.9839,-38.7273


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [23]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Create a figure to add the Google map as a layer
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))